1.1 Extraccion

In [2]:
# Carga de librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
#cargar los archivos que se van a usar
df_homicidios = pd.read_excel('./datasets/homicidios.xlsx',sheet_name = 'HECHOS',na_values=['SD', 'sd'])
df_vic_hom    = pd.read_excel('./datasets/homicidios.xlsx',sheet_name = 'VICTIMAS',na_values=['SD', 'sd'])
df_lesiones   = pd.read_excel('./datasets/lesiones.xlsx' , sheet_name = 'HECHOS',na_values=['SD', 'sd'])
df_vic_les    = pd.read_excel('./datasets/lesiones.xlsx' , sheet_name = 'VICTIMAS',na_values=['SD', 'sd'])

1.2 Transformaciones

1.2.1 Homicidios - hechos

In [4]:
#quitar las columnas que no se van a usar
df_homicidios.drop(columns=['AAAA', 'MM', 'DD', 'HORA', 'LUGAR_DEL_HECHO', 'Calle', 
                            'Altura', 'Cruce', 'Dirección Normalizada', 'PARTICIPANTES','VICTIMA'], inplace=True)

In [5]:
df_homicidios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   ID             696 non-null    object        
 1   N_VICTIMAS     696 non-null    int64         
 2   FECHA          696 non-null    datetime64[ns]
 3   HH             695 non-null    float64       
 4   TIPO_DE_CALLE  696 non-null    object        
 5   COMUNA         696 non-null    int64         
 6   XY (CABA)      696 non-null    object        
 7   pos x          696 non-null    object        
 8   pos y          696 non-null    object        
 9   ACUSADO        673 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(6)
memory usage: 54.5+ KB


In [6]:
# Cantidad de valores nulos
df_homicidios.isnull().sum()

ID                0
N_VICTIMAS        0
FECHA             0
HH                1
TIPO_DE_CALLE     0
COMUNA            0
XY (CABA)         0
pos x             0
pos y             0
ACUSADO          23
dtype: int64

In [7]:
# remplazar la comuna 0 a nulos
df_homicidios.loc[df_homicidios['COMUNA'] == '0' ,'COMUNA'] = np.nan


# remplazar los "." por nulos en latitud y longitud
df_homicidios.loc[df_homicidios['pos x'] == '.' ,'pos x'] = np.nan
df_homicidios.loc[df_homicidios['pos y'] == '.' ,'pos y'] = np.nan
df_homicidios.loc[df_homicidios['XY (CABA)'] == 'Point (. .)' ,'XY (CABA)'] = np.nan


In [8]:
# cambio a formato categórico
df_homicidios['HH'] = df_homicidios['HH'].astype('category')
df_homicidios['TIPO_DE_CALLE'] = df_homicidios['TIPO_DE_CALLE'].astype('category')
df_homicidios['COMUNA'] = df_homicidios['COMUNA'].astype('category')
df_homicidios['ACUSADO'] = df_homicidios['ACUSADO'].astype('category')

# cambio a float los valores de latitud y longitud

df_homicidios['pos x'] = df_homicidios['pos x'].astype('float')
df_homicidios['pos y'] = df_homicidios['pos y'].astype('float')




In [9]:
#renombrar las columnas
nuevos_nombres = {'ID': 'ID_HECHO','pos x':'LONGITUD','pos y':'LATITUD',
                  'HH':'HORA','ACUSADO':'VEHICULO_ACUSADO',
                  'Rango_etario ':'RANGO_ETARIO'}
                 
# Remplazar nombres
df_homicidios = df_homicidios.rename(columns=nuevos_nombres)

#agregar la columna gravedad para diferenciarlo de las lesiones
df_homicidios["GRAVEDAD"]="FATAL"

df_homicidios["FECHA"]=pd.to_datetime(df_homicidios["FECHA"], format='%y/%m/%d', errors='coerce').dt.date

In [10]:
df_homicidios.columns

Index(['ID_HECHO', 'N_VICTIMAS', 'FECHA', 'HORA', 'TIPO_DE_CALLE', 'COMUNA',
       'XY (CABA)', 'LONGITUD', 'LATITUD', 'VEHICULO_ACUSADO', 'GRAVEDAD'],
      dtype='object')

In [11]:
# Guardamos el DF listo para su análisis
df_homicidios.to_csv('./datasets/df_homicidios.csv',index=False)

1.2.2 Homicidios - victimas

In [12]:
df_vic_hom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   ROL                  706 non-null    object        
 6   VICTIMA              708 non-null    object        
 7   SEXO                 711 non-null    object        
 8   EDAD                 664 non-null    float64       
 9   FECHA_FALLECIMIENTO  649 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(5)
memory usage: 56.1+ KB


In [13]:
#eliminar las columnas que no se van  a usar
df_vic_hom.drop(columns=['AAAA', 'MM', 'DD', 'FECHA_FALLECIMIENTO'], inplace=True)
#crea un columna con la condicion
df_vic_hom['GRAVEDAD'] = 'FATAL'

In [14]:
# función que regresa 1 si la edad es menor a 18, 2 si la edad esta entre 18 y 29
# 3 si la edad esta entre 30 y 39, 4 si la edad esta entre 40 a 49 , 5 si la edad es de 50 a 59
# y 6 si la edad es igual o mayor a 60 
def rango_etario(edad):
    if edad < 18:
        return "1_ 0 a 17 años"
    elif 18 <= edad <= 29:
        return "2_ 18 a 29 años"
    elif 30 <= edad <= 39:
        return "3_ 30 a 39 años"
    elif 40 <= edad <= 49:
        return "4_ 45 a 49 años"
    elif 50 <= edad <= 59:
        return "5_ 50 a 59 años"
    elif edad >= 60:
        return "6_ 60 años o mas"
    else:
        return np.nan

In [15]:
# cambiar los SD por nan
df_vic_hom.loc[df_vic_hom['EDAD'] == 'SD', 'EDAD'] = np.nan
df_vic_hom.loc[df_vic_hom['EDAD'] == 'sd', 'EDAD'] = np.nan
#cambiar el tipo de variable
df_vic_hom['EDAD'] = df_vic_hom['EDAD'].astype('float')
#aplicar la funcion de rangos
df_vic_hom['RANGO_ETARIO'] = df_vic_hom['EDAD'].apply(lambda x: rango_etario(x))
df_vic_hom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   ID_hecho      717 non-null    object        
 1   FECHA         717 non-null    datetime64[ns]
 2   ROL           706 non-null    object        
 3   VICTIMA       708 non-null    object        
 4   SEXO          711 non-null    object        
 5   EDAD          664 non-null    float64       
 6   GRAVEDAD      717 non-null    object        
 7   RANGO_ETARIO  664 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 44.9+ KB


In [16]:
# cambiar el nomre de las columnas y estandarizarlas  
nuevos_nombres = {'ID_hecho':'ID_HECHO','ROL':'ROL_VICTIMA','VICTIMA':'VEHICULO_VICTIMA','SEXO':'SEXO_VICTIMA',"EDAD":"EDAD_VICTIMA"}
#cambiar nombre la columna edad
#df_hom_vic.rename(columns={"EDAD":"EDAD_VICTIMA"},inplace=True)
# Remplazar nombres
df_vic_hom = df_vic_hom.rename(columns=nuevos_nombres)
df_vic_hom.columns

Index(['ID_HECHO', 'FECHA', 'ROL_VICTIMA', 'VEHICULO_VICTIMA', 'SEXO_VICTIMA',
       'EDAD_VICTIMA', 'GRAVEDAD', 'RANGO_ETARIO'],
      dtype='object')

In [17]:
#cambia el ROL_VICTIMA  =  PASAJERO_ACOMPAÑANTE por PASAJERO
df_vic_hom['ROL_VICTIMA'] = df_vic_hom['ROL_VICTIMA'].replace('PASAJERO_ACOMPAÑANTE','PASAJERO')

#cambia el formato de la fecha
df_vic_hom["FECHA"]=pd.to_datetime(df_vic_hom["FECHA"], format='%y/%m/%d', errors='coerce').dt.date

In [18]:
# Guardamos el DF listo para su análisis
df_vic_hom.to_csv('./datasets/df_vic_hom.csv',index=False)

1.2.3 Lesiones - Hechos

In [19]:
df_lesiones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23785 entries, 0 to 23784
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     23785 non-null  object 
 1   n_victimas             23785 non-null  int64  
 2   aaaa                   23785 non-null  int64  
 3   mm                     23785 non-null  int64  
 4   dd                     23785 non-null  int64  
 5   fecha                  23785 non-null  object 
 6   hora                   23776 non-null  object 
 7   franja_hora            23776 non-null  float64
 8   direccion_normalizada  12917 non-null  object 
 9   comuna                 22770 non-null  object 
 10  tipo_calle             12740 non-null  object 
 11  otra_direccion         5490 non-null   object 
 12  calle                  12867 non-null  object 
 13  altura                 12771 non-null  float64
 14  cruce                  9407 non-null   object 
 15  ge

In [20]:
#eliminar las columnas que no son importantes
df_lesiones.drop(columns=['aaaa', 'mm', 'dd','hora','direccion_normalizada',
                          'calle','altura','cruce','participantes','moto',
                          'auto','transporte_publico','camion','otra_direccion',
                          'ciclista','victima'], inplace=True)

In [21]:
# Pasar las columnas a mayusculas
df_lesiones.columns = df_lesiones.columns.str.upper()

# CAMBIAR NOMBRE DE LA COLUMNA FRANJA HORARIA
df_lesiones = df_lesiones.rename(columns={'ID':'ID_HECHO'})
df_lesiones = df_lesiones.rename(columns={'FRANJA_HORA':'HORA'})
df_lesiones = df_lesiones.rename(columns={'TIPO_CALLE':'TIPO_DE_CALLE'})
df_lesiones = df_lesiones.rename(columns={'TIPO_CALLE':'TIPO_DE_CALLE'})
df_lesiones = df_lesiones.rename(columns={'GEOCODIFICACION_CABA':'XY (CABA)'})
df_lesiones = df_lesiones.rename(columns={'ACUSADO':'VEHICULO_ACUSADO'})


In [22]:
#hacer cambios a la columna fecha
df_lesiones['FECHA'].replace("'","", inplace=True)
df_lesiones['FECHA'] = df_lesiones['FECHA'].astype('datetime64[ns]')

# Remplazar "No especificada" por nulos
df_lesiones.loc[df_lesiones['COMUNA'] == 'No Especificada', 'COMUNA'] = np.nan

#Añadir gravedad
df_lesiones.loc[df_lesiones['GRAVEDAD'].isnull(), 'GRAVEDAD'] = 'LEVEMENTE HERIDO'

#cambia el formato de la fecha
df_lesiones["FECHA"]=pd.to_datetime(df_lesiones["FECHA"], format='%y/%m/%d', errors='coerce').dt.date

In [23]:
df_lesiones.columns

Index(['ID_HECHO', 'N_VICTIMAS', 'FECHA', 'HORA', 'COMUNA', 'TIPO_DE_CALLE',
       'XY (CABA)', 'LONGITUD', 'LATUTID', 'VEHICULO_ACUSADO', 'GRAVEDAD'],
      dtype='object')

In [24]:
# Guardamos el DF listo para su análisis
df_lesiones.to_csv('./datasets/df_lesiones.csv',index=False)

1.2.4 Lesiones - Victimas

In [25]:
# Pasar las columnas a mayusculas
df_vic_les.columns = df_vic_les.columns.str.upper()

# Cambiar nombres a las columnas para coincidir con el df_homicidios
df_vic_les = df_vic_les.rename(columns={'ID HECHO':'ID_HECHO','FECHA ':'FECHA','SEXO':'SEXO_VICTIMA'})

#cambia el formato de la fecha
df_vic_les["FECHA"]=pd.to_datetime(df_vic_les["FECHA"], format='%y/%m/%d', errors='coerce').dt.date

df_vic_les.columns

Index(['ID_HECHO', 'AAA', 'MM', 'DD', 'FECHA', 'VEHICULO_VICTIMA',
       'SEXO_VICTIMA', 'EDAD_VICTIMA', 'GRAVEDAD'],
      dtype='object')

In [26]:
#eliminar las columnas que no se van a usar
df_vic_les.drop(columns=['AAA', 'MM', 'DD'], inplace=True)

# Columnas
df_vic_les.columns

Index(['ID_HECHO', 'FECHA', 'VEHICULO_VICTIMA', 'SEXO_VICTIMA', 'EDAD_VICTIMA',
       'GRAVEDAD'],
      dtype='object')

In [27]:
# Creamos la columna ROL_VICTIMA con nulls
df_vic_les['ROL_VICTIMA'] = np.nan

# remplazar los nulos de la columna GRAVEDAD por "LEVEMENTE HERIDO"
df_vic_les.loc[df_vic_les['GRAVEDAD'].isnull(), 'GRAVEDAD'] = 'LEVEMENTE HERIDO'


In [28]:
# Rango Etario
df_vic_les['EDAD_VICTIMA'] = df_vic_les['EDAD_VICTIMA'].astype('float')
df_vic_les['RANGO_ETARIO'] = df_vic_les['EDAD_VICTIMA'].apply(lambda x: rango_etario(x))

In [29]:
# homogenizar en FEMENINO y MASCULINO
df_vic_les.loc[df_vic_les['SEXO_VICTIMA'] == 'MUJER', 'SEXO_VICTIMA'] = 'FEMENINO'
df_vic_les.loc[df_vic_les['SEXO_VICTIMA'] == 'Mujer', 'SEXO_VICTIMA'] = 'FEMENINO'
df_vic_les.loc[df_vic_les['SEXO_VICTIMA'] == 'mujer ', 'SEXO_VICTIMA'] = 'FEMENINO'
df_vic_les.loc[df_vic_les['SEXO_VICTIMA'] == 'mujer', 'SEXO_VICTIMA'] = 'FEMENINO'
df_vic_les.loc[df_vic_les['SEXO_VICTIMA'] == 'Mujer ', 'SEXO_VICTIMA'] = 'FEMENINO'
df_vic_les.loc[df_vic_les['SEXO_VICTIMA'] == 'Varon', 'SEXO_VICTIMA'] = 'MASCULINO'
df_vic_les.loc[df_vic_les['SEXO_VICTIMA'] == 'varon', 'SEXO_VICTIMA'] = 'MASCULINO'

In [30]:
# inprime valores unicos de la columna SEXO_VICTIMA
df_vic_les['SEXO_VICTIMA'].unique()

array(['MASCULINO', nan, 'FEMENINO'], dtype=object)

In [31]:
# Guardamos el DF listo para su análisis
df_vic_les.to_csv('./datasets/df_vic_les.csv',index=False)

In [32]:
df_homicidios.tail(5)


,ID_HECHO,N_VICTIMAS,FECHA,HORA,TIPO_DE_CALLE,COMUNA,XY (CABA),LONGITUD,LATITUD,VEHICULO_ACUSADO,GRAVEDAD
691,2021-0093,1,2021-12-13,17.0,AVENIDA,7.0,Point (102728.60090138 98186.24929177),-58.433538,-34.645616,AUTO,FATAL
692,2021-0094,1,2021-12-20,1.0,AUTOPISTA,9.0,Point (99624.29795829 97569.69801131),-58.467398,-34.651178,AUTO,FATAL
693,2021-0095,1,2021-12-30,0.0,AVENIDA,11.0,Point (99116.45492358 101045.23284826),-58.472934,-34.619847,CARGAS,FATAL
694,2021-0096,1,2021-12-15,10.0,AVENIDA,9.0,Point (99324.54463985 97676.26932409),-58.470668,-34.650217,CARGAS,FATAL
695,2021-0097,1,2021-11-18,6.0,CALLE,1.0,Point (107664.16647795 104708.63962087),-58.379762,-34.586796,AUTO,FATAL


In [33]:
df_lesiones.tail(5)

,ID_HECHO,N_VICTIMAS,FECHA,HORA,COMUNA,TIPO_DE_CALLE,XY (CABA),LONGITUD,LATUTID,VEHICULO_ACUSADO,GRAVEDAD
23780,LC-2021-0652849,1,2021-12-31,19.0,9,AVENIDA,Point (95400.5441905024 96621.6007711877),-5.851348e+01,-3.465971e+01,NaN,LEVEMENTE HERIDO
23781,LC-2021-0652865,2,2021-12-31,19.0,9,AVENIDA,Point (97705.4491469278 98614.8825470092),-5.848833e+01,-3.464175e+01,NaN,LEVEMENTE HERIDO
23782,LC-2021-0652907,1,2021-12-31,20.0,1,NaN,Point (107377.10694920316745993 105120.7873965...,-5.838289e+01,-3.458308e+01,NaN,LEVEMENTE HERIDO
23783,LC-2021-0652921,1,2021-12-31,22.0,5,CALLE,Point (104472.702045632 101660.930364258),-5.841453e+01,-3.461429e+01,TRANSPORTE PUBLICO,GRAVE
23784,LC-2020-0244615,1,2020-05-17,21.0,NaN,NaN,Point (114990.04979284359433223 88934.67755418...,-5.829963e+09,-3.472891e+09,AUTO,LEVEMENTE HERIDO


In [117]:
df_vic_les["ROL_VICTIMA"]

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
27600   NaN
27601   NaN
27602   NaN
27603   NaN
27604   NaN
Name: ROL_VICTIMA, Length: 27605, dtype: float64